In [2]:
!pip install tensorflow
!pip install keras
!pip install scikeras
!pip install xgboost


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasClassifier
import xgboost as xgb

2024-12-08 10:13:08.988436: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-08 10:13:09.212254: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-08 10:13:09.404760: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733652789.695985   84293 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733652789.806355   84293 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-08 10:13:10.477023: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [4]:
data = pd.read_csv('./cleaned_data_2.csv')

In [5]:
X = data.drop('readmitted', axis=1)
y = data['readmitted']

#TRUE/FALSE becomes 1/0
X = X.replace({'TRUE': 1, 'FALSE': 0})

In [6]:
# Identify categorical columns
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

# Apply Label Encoding to categorical variables
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    label_encoders[col] = le

In [7]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# SMOTE to handle imbalanced samples
X_train_resampled, X_test, y_train_resampled, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.2, stratify=y_resampled, random_state=42
)

In [8]:
# Design the autoencoder
input_dim = X_train_resampled.shape[1]
encoding_dim = input_dim // 2  # Adjust this as needed

# Autoencoder architecture
input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation='relu')(input_layer)
decoder = Dense(input_dim, activation='relu')(encoder)

autoencoder = Model(inputs=input_layer, outputs=decoder)
autoencoder.compile(optimizer=Adam(), loss='mse')

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
# Train the autoencoder
autoencoder.fit(
    X_train_resampled, X_train_resampled,
    epochs=50,
    batch_size=32,
    shuffle=True,
    callbacks=[early_stop],
    validation_data=(X_test, X_test)
)

# Extract encoded features
encoder_model = Model(inputs=input_layer, outputs=encoder)
X_train_encoded = encoder_model.predict(X_train_resampled)
X_test_encoded = encoder_model.predict(X_test)

2024-12-08 10:13:22.583635: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
2024-12-08 10:13:22.711567: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 39692328 exceeds 10% of free system memory.
2024-12-08 10:13:22.755415: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 39692328 exceeds 10% of free system memory.


Epoch 1/50
3976/3976 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - loss: 553.4520 - val_loss: 0.5904
Epoch 2/50
3976/3976 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 0.5822 - val_loss: 0.5697
Epoch 3/50
3976/3976 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - loss: 0.5710 - val_loss: 0.5691
Epoch 4/50
3976/3976 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step - loss: 0.5733 - val_loss: 0.5688
Epoch 5/50
3976/3976 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 0.5707 - val_loss: 0.5697
Epoch 6/50
3976/3976 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - loss: 0.5704 - val_loss: 0.6173
Epoch 7/50
3976/3976 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - loss: 0.5735 - val_loss: 0.5692
Epoch 8/50
3976/3976 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - loss: 0.5704 - val_loss: 0.5679
Epoch 9/50
3976/3976 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 0.5731 - val_loss: 0.5679
Epoch 10/50
3976/3976 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - loss: 0.5708 - val_loss: 0.5719
Epoch 11/50
3976/3976 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - loss: 0.5711 - val_loss: 0.5678
Epoch 12/50
3976/3976 ━

In [9]:
#Large and Thought Out Grid Search
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 8, 10],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

xgb_clf = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss'
)

In [10]:
warnings.filterwarnings("ignore", message=".*Parameters.*")

grid_search = GridSearchCV(
    estimator=xgb_clf,
    param_grid=param_grid,
    scoring='f1',
    cv=3,
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train_encoded, y_train_resampled)

# Best model
best_xgb = grid_search.best_estimator_
best_params = grid_search.best_params_

print("Best Parameters from Grid Search:")
print(best_params)

Fitting 3 folds for each of 144 candidates, totalling 432 fits


/usr/local/python/3.12.1/lib/python3.12/site-packages/numpy/ma/core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best Parameters from Grid Search:
{'colsample_bytree': 0.8, 'learning_rate': 0.3, 'max_depth': 10, 'n_estimators': 300, 'subsample': 0.8}


In [11]:
y_pred = best_xgb.predict(X_test_encoded)
y_pred_proba = best_xgb.predict_proba(X_test_encoded)[:, 1]

In [14]:
onf_matrix = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

# Print evaluation metrics
# print("Confusion Matrix:")
# print(conf_matrix)
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")
print(f"AUC-ROC: {roc_auc:.2f}")

Accuracy: 0.85
Precision: 0.83
Recall: 0.89
F1-Score: 0.86
AUC-ROC: 0.93
